In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F
from string import printable
import numpy as np


In [ ]:
class CharLSTM(nn.Module):
    def __init__(self, hidden_size=256, n_hidden_layers=3, dropout_prob=0.5):
        super().__init__()
        self.legal_chars = printable
        self.hidden_size = hidden_size
        self.n_hidden_layers = n_hidden_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.lstm_layer = nn.LSTM(
            len(self.legal_chars), 
            hidden_size, 
            n_hidden_layers, 
            dropout=dropout_prob, 
            batch_first=True
        )
        self.dropout_layer = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, len(self.legal_chars))
        self.to(self.device)

    def forward(self, x, hidden):
        x = x.to(self.device)
        lstm_output, hidden = self.lstm_layer(x, hidden)
        output = self.dropout_layer(lstm_output)
        output = output.contiguous().view(-1, self.hidden_size)
        return self.fc(output), hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_hidden_layers, batch_size, self.hidden_size).zero_().to(self.device),
                  weight.new(self.n_hidden_layers, batch_size, self.hidden_size).zero_().to(self.device))
        return hidden

In [ ]:
SEQ_LENGTH = 100
BATCH_SIZE = 128
int_to_char = dict(enumerate(printable))
char_to_int = {char:int_ for int_, char in int_to_char.items()}
file_path = '/content/drive/My Drive/Colab Notebooks/Practice/CharacterGeneration/file.txt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(file_path, 'r') as f:
    text = f.read()

print(text[:130])
encoded = np.array([char_to_int[ch] for ch in text])
print()
print(encoded[:130])

The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan 

[55 17 14 94 51 27 24 19 14 12 29 94 42 30 29 14 23 11 14 27 16 94 40 37
 24 24 20 94 24 15 94 55 17 14 94 36 13 31 14 23 29 30 27 14 28 94 24 15
 94 54 17 14 27 21 24 12 20 94 43 24 21 22 14 28 96 11 34 94 54 18 27 94
 36 27 29 17 30 27 94 38 24 23 10 23 94 39 24 34 21 14 96 69 64  1  5 94
 18 23 94 24 30 27 94 28 14 27 18 14 28 94 11 34 94 54 18 27 94 36 27 29
 17 30 27 94 38 24 23 10 23 94]


In [ ]:
def one_hot_encoder(arr):
    """
    Input list_ contains indices of elements containing 1
    """
    n_labels = len(printable)
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


def create_batches(arr=encoded, batch_size=BATCH_SIZE, sequence_length=SEQ_LENGTH):
    arr_len = len(arr)
    num_batches = arr_len // (batch_size * sequence_length)

    arr = arr[:num_batches * batch_size*sequence_length]
    arr = arr.reshape((num_batches, batch_size, sequence_length))
    for i in range(0, num_batches):
        x = arr[i]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:, :-1] = x[:, 1:]
        try:
            y[:, -1] = arr[i + 1][:, 0]
            yield x, y
        except Exception as e:
            print('Exception at index:', i)

batches = create_batches()
x, y = next(batches)
x

NameError: ignored

In [ ]:
model = CharLSTM()
model

CharLSTM(
  (lstm_layer): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.5)
  (dropout_layer): Dropout(p=0.5)
  (fc): Linear(in_features=256, out_features=100, bias=True)
)

In [ ]:
def train(net=model, data=encoded, epochs=20, batch_size=BATCH_SIZE, seq_length=SEQ_LENGTH, lr=0.001, clip=5, print_every=100):
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    counter = 0
    for epoch in range(epochs):
        h = net.init_hidden(batch_size)

        for x, y in create_batches():
            counter += 1
            x = one_hot_encoder(x)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            targets = targets.to(device)
            h = tuple([each.data for each in h])
            opt.zero_grad()
            output, h = net(inputs, h)
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            if counter % print_every == 0:
                print("Epoch: {}/{}...".format(epoch+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()))
                





In [ ]:
train(epochs=50)

Epoch: 1/50... Step: 100... Loss: 1.4962...
Epoch: 1/50... Step: 200... Loss: 1.4632...
Epoch: 1/50... Step: 300... Loss: 1.4616...
Epoch: 1/50... Step: 400... Loss: 1.4414...
Exception at index: 494
Epoch: 2/50... Step: 500... Loss: 1.5595...
Epoch: 2/50... Step: 600... Loss: 1.5143...
Epoch: 2/50... Step: 700... Loss: 1.3982...
Epoch: 2/50... Step: 800... Loss: 1.5132...
Epoch: 2/50... Step: 900... Loss: 1.4703...
Exception at index: 494
Epoch: 3/50... Step: 1000... Loss: 1.4686...
Epoch: 3/50... Step: 1100... Loss: 1.5354...
Epoch: 3/50... Step: 1200... Loss: 1.4487...
Epoch: 3/50... Step: 1300... Loss: 1.4907...
Epoch: 3/50... Step: 1400... Loss: 1.4373...
Exception at index: 494
Epoch: 4/50... Step: 1500... Loss: 1.5473...
Epoch: 4/50... Step: 1600... Loss: 1.5278...
Epoch: 4/50... Step: 1700... Loss: 1.4304...
Epoch: 4/50... Step: 1800... Loss: 1.4705...
Epoch: 4/50... Step: 1900... Loss: 1.4154...
Exception at index: 494
Epoch: 5/50... Step: 2000... Loss: 1.5524...
Epoch: 5/50..

In [ ]:
def save_weights(model, path):
    torch.save(model.state_dict(), path)
    
def load_weights(model, path, test=False):
    model.load_state_dict(torch.load(path))
    if test:
        model.eval()
    else:
        model.train()

In [ ]:
save_weights(model, '/content/drive/My Drive/Colab Notebooks/Practice/CharacterGeneration/model.pth')

In [ ]:
train_on_gpu = True
def predict(net, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''
    net.eval()
    # tensor inputs
    x = np.array([[char_to_int[char]]])
    x = one_hot_encoder(x)
    inputs = torch.from_numpy(x)
    
    if(train_on_gpu):
        inputs = inputs.cuda()
    
    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if(train_on_gpu):
        p = p.cpu() # move to cpu
    
    # get top characters
    if top_k is None:
        top_ch = np.arange(len(printable))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()
    
    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p/p.sum())
    
    # return the encoded value of the predicted char and the hidden state
    return int_to_char[char], h

def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(model, 1000, prime='They', top_k=5))

They will not always be the chest. The
clothing of the station of the same conception of something
and the more
and the candles in a present, and his husband
had surprised and
say and so that he seemed a communication in the
command of the morning of
the first and character. Hisstrength of the
campaign to the same
side of its father. He says something at the freedom and son, with a smell of horses at all a servants the
same changing
his, he as it may
have seen to
his finger and striking such power of their patted to several pattend in the position of
the
first period. And any store of
herself were askants. His faces
which she had no
son instead of significance of striking how he was strange, and that such a music carried
and a little still
son, and he
had been
as his ships, the movement is
not a present to him to
be desperately said that they were not
the
country to a present of her house.

"A character, and the son and
some officers and hussars and told the prisoners, how if was had s